In [1]:
import pandas as pd
import numpy as np
import collections
import time
import math

In [2]:
#                            ,usecols=['item_id','n_clicks','views','clicks_views_ratio','sum_of_properties',
#                                    'Satisfactory Rating', 'Shower','Washing Machine', 'Fridge','3 Star',
#                                     'Air Conditioning','Good Rating','Car Park' ]
item_metadata = pd.read_csv('D:\\Dokumenty\\Systemy_rekomendacyjne\\data\\item_metadata_all.csv',
                           sep=','
                           ,usecols=['item_id','n_clicks','views','clicks_views_ratio','sum_of_properties',
                                   'Satisfactory Rating','Good Rating','Very Good Rating', 'Excellent Rating',
                                    '1 Star','2 Star','3 Star','4 Star','5 Star']
                           )
item_metadata['item_id'] = item_metadata['item_id'].astype(int)
item_metadata.shape

(813442, 14)

In [3]:
train = pd.read_csv('D:\\Dokumenty\\Systemy_rekomendacyjne\\data\\train.csv',
                       sep=',',
                    skiprows=range(1, 10000000)
#                    nrows=100000
                   )
train = train[train["action_type"] == "clickout item"]
print(train.shape)
train[:3]

(267552, 13)


,index,session_id,timestamp,user_id,step,action_type,reference,platform,city,device,current_filters,impressions,prices
16,10000015,99af55eb223be,1541371324,0VJVWTT13VW2,3,clickout item,8924918,ES,"Badajoz, Spain",mobile,NaN,40053|80867|147423|100365|39980|100366|1455161...,56|59|78|66|50|53|38|52|52|39|40|28|37|45|49|3...
138,10000137,333e4d0847e8c,1541371324,H3V8YFGBQ7GN,1,clickout item,5385,DE,"Nuremberg, Germany",desktop,NaN,5385|5372|3499432|5962480|5380|5967884|51576|5...,121|223|184|138|159|187|135|141|175|156|267|12...
152,10000151,b6abe531702df,1541371325,9Q3WSMO62M5Q,162,clickout item,7990,UK,"San Francisco, USA",desktop,5 Star,14900|7990|982947|101476|60287|357256|7993|457...,364|228|939|498|323|162|120|95|180|111|83|132|...


In [4]:
def fill_to_25_rows(df):
    impressions_str = str(df['impressions'])
    prices_str = str(df['prices'])
    count_of_impressions = impressions_str.count('|') + 1
    
    count_of_missing_impresions = 25 - count_of_impressions
    while count_of_missing_impresions != 0:
        impressions_str += '|-1'
        count_of_missing_impresions += -1
    
    df['impressions'] = impressions_str
    df['prices'] = impressions_str
    return df

# train = train.apply(fill_to_25_rows, axis=1)

In [5]:
def string_to_array(s):
    """Convert pipe separated string to array."""

    if isinstance(s, str):
        out = s.split("|")
    elif math.isnan(s):
        out = []
    else:
        raise ValueError("Value must be either string of nan")
    return out

def explode_impressions_and_prices(df_in):
    """Explode column col_expl of array type into multiple rows."""

    df = df_in.copy()
    df.loc[:, 'impressions'] = df['impressions'].apply(string_to_array)  # zamienia 1|2|3 na [1,2,3]
    df.loc[:, 'prices'] = df['prices'].apply(string_to_array)  # zamienia 1|2|3 na [1,2,3]

    df = df.sort_values(by=['session_id'])
    df['number_of_impressions'] = df['impressions'].apply(len)
    
    df_out = pd.DataFrame(
        {col: np.repeat(df[col].values,
                        df['impressions'].str.len())
         for col in df.columns.drop('impressions')}
    )
    

    df_out.loc[:, 'impressions'] = np.concatenate(df['impressions'].values)
    df_out.loc[:, 'impressions'] = df_out['impressions'].apply(int)
    
    df_out.loc[:, 'prices'] = np.concatenate(df['prices'].values)
    df_out.loc[:, 'prices'] = df_out['prices'].apply(int)

    return df_out

In [6]:
train = explode_impressions_and_prices(train)
train[:3]

,index,session_id,timestamp,user_id,step,action_type,reference,platform,city,device,current_filters,prices,number_of_impressions,impressions
0,12440114,000138ab4f789,1541447064,WDBP226XSN9E,26,clickout item,2423456,US,"Fort Lauderdale, USA",desktop,Hotel|Motel|Resort|Airport Shuttle|Best Value,157,25,224686
1,12440114,000138ab4f789,1541447064,WDBP226XSN9E,26,clickout item,2423456,US,"Fort Lauderdale, USA",desktop,Hotel|Motel|Resort|Airport Shuttle|Best Value,112,25,148677
2,12440114,000138ab4f789,1541447064,WDBP226XSN9E,26,clickout item,2423456,US,"Fort Lauderdale, USA",desktop,Hotel|Motel|Resort|Airport Shuttle|Best Value,140,25,2653512


In [7]:
train['position'] = train.groupby(['user_id','timestamp','session_id','step']).cumcount()+1
train[:3]

,index,session_id,timestamp,user_id,step,action_type,reference,platform,city,device,current_filters,prices,number_of_impressions,impressions,position
0,12440114,000138ab4f789,1541447064,WDBP226XSN9E,26,clickout item,2423456,US,"Fort Lauderdale, USA",desktop,Hotel|Motel|Resort|Airport Shuttle|Best Value,157,25,224686,1
1,12440114,000138ab4f789,1541447064,WDBP226XSN9E,26,clickout item,2423456,US,"Fort Lauderdale, USA",desktop,Hotel|Motel|Resort|Airport Shuttle|Best Value,112,25,148677,2
2,12440114,000138ab4f789,1541447064,WDBP226XSN9E,26,clickout item,2423456,US,"Fort Lauderdale, USA",desktop,Hotel|Motel|Resort|Airport Shuttle|Best Value,140,25,2653512,3


In [8]:
# # DODANIE CITY I COUNTRY
# city, country = (train['city']
#                     .values[0]
#                     .replace(" ", "")
#                     .split(','))
# print(city)
# print(country)

# cities = []
# countries = []
# l = []
# for index in range(len(train['city'].values)):
#     l = (train['city']
#                     .values[index]
#                     .replace(" ", "")
#                     .split(','))
#     cities.append(l[0])
#     countries.append(l[1])
# #     if len(l) > 2 :
# #         print(l[0])
# #         print(l[1])
# #         print(l[2])
# #         print(len(l))

# train = train.drop(['city'], axis=1)
# train['city'] = cities
# train['country'] = countries
# train[:3]

In [9]:
def mean_price(df):
    return df['prices'].mean()
df_mean_price = train.groupby(['session_id', 'timestamp']).apply(lambda df: df['prices'].mean()).to_frame("mean_price")
df_mean_price[:3]
train = train.merge(df_mean_price,
                     left_on=['session_id', 'timestamp'],
                     right_on=['session_id', 'timestamp'],
                     how='left')
train['mean_price_difference'] = abs(train['prices'] - train['mean_price'])
train[:3]

,index,session_id,timestamp,user_id,step,action_type,reference,platform,city,device,current_filters,prices,number_of_impressions,impressions,position,mean_price,mean_price_difference
0,12440114,000138ab4f789,1541447064,WDBP226XSN9E,26,clickout item,2423456,US,"Fort Lauderdale, USA",desktop,Hotel|Motel|Resort|Airport Shuttle|Best Value,157,25,224686,1,192.24,35.24
1,12440114,000138ab4f789,1541447064,WDBP226XSN9E,26,clickout item,2423456,US,"Fort Lauderdale, USA",desktop,Hotel|Motel|Resort|Airport Shuttle|Best Value,112,25,148677,2,192.24,80.24
2,12440114,000138ab4f789,1541447064,WDBP226XSN9E,26,clickout item,2423456,US,"Fort Lauderdale, USA",desktop,Hotel|Motel|Resort|Airport Shuttle|Best Value,140,25,2653512,3,192.24,52.24


In [10]:
train = train.merge(item_metadata,
                   left_on='impressions',
                   right_on='item_id',
                   how='left')
train[:3]

,index,session_id,timestamp,user_id,step,action_type,reference,platform,city,device,...,1 Star,2 Star,Satisfactory Rating,4 Star,Excellent Rating,Very Good Rating,Good Rating,5 Star,3 Star,sum_of_properties
0,12440114,000138ab4f789,1541447064,WDBP226XSN9E,26,clickout item,2423456,US,"Fort Lauderdale, USA",desktop,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,52.0
1,12440114,000138ab4f789,1541447064,WDBP226XSN9E,26,clickout item,2423456,US,"Fort Lauderdale, USA",desktop,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34.0
2,12440114,000138ab4f789,1541447064,WDBP226XSN9E,26,clickout item,2423456,US,"Fort Lauderdale, USA",desktop,...,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,39.0


In [11]:
train = train.rename({'index':'item_id'})
train = train[["item_id", "reference",'n_clicks','views','clicks_views_ratio','position','mean_price','mean_price_difference',
               'Satisfactory Rating','Good Rating','Very Good Rating', 'Excellent Rating',
                '1 Star','2 Star','3 Star','4 Star','5 Star',
              ]]
# train = train[["item_id", "reference",'n_clicks','views','clicks_views_ratio','sum_of_properties','mean_price','mean_price_difference','position']]
train[:3]

,item_id,reference,n_clicks,views,clicks_views_ratio,position,mean_price,mean_price_difference,Satisfactory Rating,Good Rating,Very Good Rating,Excellent Rating,1 Star,2 Star,3 Star,4 Star,5 Star
0,224686,2423456,26.0,353,0.073654,1,192.24,35.24,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,148677,2423456,7.0,167,0.041916,2,192.24,80.24,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,2653512,2423456,23.0,530,0.043396,3,192.24,52.24,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


In [12]:
train['reference'] = train['reference'].astype(int)
train['clicked'] = np.where(train['item_id'] == train['reference'], 1, 0)
train[:3]

,item_id,reference,n_clicks,views,clicks_views_ratio,position,mean_price,mean_price_difference,Satisfactory Rating,Good Rating,Very Good Rating,Excellent Rating,1 Star,2 Star,3 Star,4 Star,5 Star,clicked
0,224686,2423456,26.0,353,0.073654,1,192.24,35.24,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
1,148677,2423456,7.0,167,0.041916,2,192.24,80.24,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
2,2653512,2423456,23.0,530,0.043396,3,192.24,52.24,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0


In [13]:
# usuwa zduplikowane hotele
# train.drop_duplicates('item_id', inplace=True)
# collections.Counter(train['item_id'].values)

In [14]:
# collections.Counter(train['clicked'].values)

In [15]:
# import plotly.express as px
# fig = px.scatter(train[:10000],x='n_clicks',y='views', color='clicked')
# fig.show()

In [16]:
train = train.drop(columns=['reference','mean_price'])
train[:3]

,item_id,n_clicks,views,clicks_views_ratio,position,mean_price_difference,Satisfactory Rating,Good Rating,Very Good Rating,Excellent Rating,1 Star,2 Star,3 Star,4 Star,5 Star,clicked
0,224686,26.0,353,0.073654,1,35.24,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
1,148677,7.0,167,0.041916,2,80.24,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
2,2653512,23.0,530,0.043396,3,52.24,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0


In [17]:
# train['Rating'] = train['Satisfactory Rating']*1 + train['Good Rating']*2 + train['Very Good Rating']*3+train['Excellent Rating']*4
train['Stars'] = train['1 Star']*1 + train['2 Star']*2 + train['3 Star']*3 + train['4 Star']*4 + train['5 Star']*5

train['Rating'] = 0
train.loc[train['Satisfactory Rating']==1, 'Rating'] = 1
train.loc[train['Good Rating']==1, 'Rating'] = 2
train.loc[train['Very Good Rating']==1, 'Rating'] = 3
train.loc[train['Excellent Rating']==1, 'Rating'] = 4


train[:3]

,item_id,n_clicks,views,clicks_views_ratio,position,mean_price_difference,Satisfactory Rating,Good Rating,Very Good Rating,Excellent Rating,1 Star,2 Star,3 Star,4 Star,5 Star,clicked,Stars,Rating
0,224686,26.0,353,0.073654,1,35.24,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,2.0,2
1,148677,7.0,167,0.041916,2,80.24,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,2.0,0
2,2653512,23.0,530,0.043396,3,52.24,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0,2.0,3


In [18]:
# train[pd.isnull(train).any(axis=1)]

# Dane

In [ ]:
# train.loc[train['clicked']==1][['Rating','Stars']].mean()

In [ ]:
import plotly.graph_objects as go
di = dict(collections.Counter(train.loc[train['clicked']==1]['Stars']))
fig = go.Figure(
    data=[go.Bar(y=list(di.values())[:6], x=list(di.keys())[:6])],
    layout_title_text="Liczba gwiazdek dla kliknięć"
)
fig.show()

In [ ]:
di = dict(collections.Counter(train.loc[train['clicked']==0]['Stars']))
fig = go.Figure(
    data=[go.Bar(y=list(di.values())[:6], x=list(di.keys())[:6])],
    layout_title_text="Liczba gwiazdek dla nieklikniętych"
)
fig.show()

In [ ]:
di = dict(collections.Counter(train.loc[train['clicked']==1]['Rating']))
fig = go.Figure(
    data=[go.Bar(y=list(di.values())[:5], x=list(di.keys())[:5])],
    layout_title_text="Liczba ratingu dla kliknięć"
)
fig.show()

In [ ]:
di = dict(collections.Counter(train.loc[train['clicked']==0]['Rating']))
fig = go.Figure(
    data=[go.Bar(y=list(di.values())[:5], x=list(di.keys())[:5])],
    layout_title_text="Liczba ratingu dla nieklikniętych"
)
fig.show()

In [ ]:
# import plotly.express as px
# fig = px.scatter(train[:1000],x='Rating',y='Stars', color='clicked')
# fig.show()

# Sieć neuronowa

In [ ]:
# 'Satisfactory Rating','Shower','Washing Machine', 'Fridge','3 Star','Air Conditioning','Good Rating','Car Park'
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train[['item_id','mean_price_difference','n_clicks','clicks_views_ratio','position','Rating','Stars']], 
                                                    train['clicked'],
                                                    test_size=0.2,
                                                    random_state=0)
X_train[:3]

In [ ]:
X_train_id_columns = X_train.reset_index().drop(columns=['index'], axis=1)['item_id']
X_train_rest_columns = X_train.reset_index().drop(columns=['index'], axis=1)[['mean_price_difference','n_clicks','clicks_views_ratio','position','Rating','Stars']]

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(X_train_rest_columns)
X_train_rest_columns = sc.transform(X_train_rest_columns)
# X_test = sc.transform(X_test)
X_train_rest_columns[:3]

In [ ]:
X_train = np.hstack((X_train_id_columns.values.reshape(-1,1), X_train_rest_columns)) # .values.reshape(-1,1)
X_train

In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation

In [ ]:
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=7))
# model.add(Dense(70, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',f1_m,precision_m, recall_m])
history = model.fit(X_train, y_train.values, epochs=3, batch_size=1025)

In [ ]:
loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, y_test, verbose=1)


### SGDClassifier

In [19]:
from sklearn.model_selection import train_test_split
train = train.fillna(-1)
X_train, X_test, y_train, y_test = train_test_split(train[['item_id','mean_price_difference','n_clicks','clicks_views_ratio','position','Rating','Stars']], 
                                                    train['clicked'],
                                                    test_size=0.2,
                                                    random_state=0)
X_train[:3]

,item_id,mean_price_difference,n_clicks,clicks_views_ratio,position,Rating,Stars
2416764,40894,69.16,16.0,0.031068,2,3,4.0
1560117,46725,5.48,3.0,0.042857,21,2,1.0
2725736,16758,49.72,5.0,0.072464,23,4,2.0


In [28]:
from sklearn.linear_model import SGDClassifier
classifier = SGDClassifier(n_jobs=-1, loss='log', class_weight={0.0:1/25,1.0:24/25})
classifier = classifier.fit(X_train[:25], y_train[:25])
counter = 25

while len(y_train) - 25 > counter:
    counter += 25
    classifier = classifier.partial_fit(X_train[counter:counter+25], y_train[counter:counter+25])



In [29]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, balanced_accuracy_score

y_pred = classifier.predict(X_test)

print(collections.Counter(y_test))
print(collections.Counter(y_pred))

print('balanced_accuracy_score: {0}'.format(balanced_accuracy_score(y_test, y_pred)))
print('accuracy_score: {0}'.format(accuracy_score(y_test, y_pred)))
print('Nieprawidłowo sklasyfikowane próbki: %d' % (y_test != y_pred).sum())
print('classification_report :\n', classification_report(y_test, y_pred))
confmat = confusion_matrix(y_true=y_test, y_pred=y_pred)
print(confmat)

Counter({0: 1178172, 1: 53213})
Counter({1: 1231385})
balanced_accuracy_score: 0.5
accuracy_score: 0.0432139420246308
Nieprawidłowo sklasyfikowane próbki: 1178172


D:\Instalowane\Programy\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


classification_report :
               precision    recall  f1-score   support

           0       0.00      0.00      0.00   1178172
           1       0.04      1.00      0.08     53213

    accuracy                           0.04   1231385
   macro avg       0.02      0.50      0.04   1231385
weighted avg       0.00      0.04      0.00   1231385

[[      0 1178172]
 [      0   53213]]


# NAUKA

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression, Perceptron, SGDClassifier
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, balanced_accuracy_score
from sklearn.preprocessing import StandardScaler, normalize
from imblearn.under_sampling import RandomUnderSampler

In [ ]:
nans = lambda df: df[df.isnull().any(axis=1)]
print(len(train))
train = train.dropna()
print(len(train))

In [ ]:
train[:3]

In [ ]:
# 'n_clicks','views','clicks_views_ratio', 'mean_price','mean_price_difference','position'
X_train, X_test, y_train, y_test = train_test_split(train[['mean_price_difference','n_clicks','clicks_views_ratio','position','Rating','Stars']], 
                                                    train['clicked'],
                                                    test_size=0.2,
                                                    random_state=0)
# rus = RandomUnderSampler(random_state=0)
# X_train, y_train = rus.fit_resample(X_train, y_train)

In [ ]:
sc = StandardScaler()
sc.fit(X_train)
X_train = sc.transform(X_train)
# X_train = normalize(X_train, norm='l2')
X_test = sc.transform(X_test)
# X_test = normalize(X_test, norm='l2')

In [ ]:
collections.Counter(y_train)

In [ ]:
from sklearn.linear_model import LogisticRegression
# logreg = DecisionTreeClassifier(class_weight={0.0:0.8, 1.0:0.2})
logreg = LogisticRegression(n_jobs=-1, class_weight={0.0:0.2, 1.0:0.8}) #class_weight={0.0:0.2, 1.0:0.8}
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
# y_pred_prob = logreg.predict_proba(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, balanced_accuracy_score

print(collections.Counter(y_test))
print(collections.Counter(y_pred))

print('balanced_accuracy_score: {0}'.format(balanced_accuracy_score(y_test, y_pred)))
print('accuracy_score: {0}'.format(accuracy_score(y_test, y_pred)))
print('Nieprawidłowo sklasyfikowane próbki: %d' % (y_test != y_pred).sum())
print('classification_report :\n', classification_report(y_test, y_pred))
confmat = confusion_matrix(y_true=y_test, y_pred=y_pred)
print(confmat)

# ZBIÓR TESTOWY

In [ ]:
test = pd.read_csv('D:\\Dokumenty\\Systemy_rekomendacyjne\\data\\test.csv',
                       sep=',',
#                     skiprows=range(1, 11000000)
#                    nrows=10000
                   )
# test = test[test["action_type"] == "clickout item" & test['reference'].isnull()]
mask = test["reference"].isnull() & (test["action_type"] == "clickout item")
test = test[mask]
# print(test.shape)
test[:3]

In [ ]:
test = explode_impressions_and_prices(test)
test[:3]

In [ ]:
test['position'] = test.groupby(['user_id','timestamp','session_id','step']).cumcount()+1
test[:3]

In [ ]:
index_df = test[["user_id","session_id","timestamp","step","impressions"]]
index_df[:10]

In [ ]:
df_mean_price = test.groupby(['session_id', 'timestamp']).apply(lambda df: df['prices'].mean()).to_frame("mean_price")
df_mean_price[:3]
test = test.merge(df_mean_price,
                     left_on=['session_id', 'timestamp'],
                     right_on=['session_id', 'timestamp'],
                     how='left')
test['mean_price_difference'] = abs(test['prices'] - test['mean_price'])
test[:3]

In [ ]:
test = test.merge(item_metadata,
                   left_on='impressions',
                   right_on='item_id',
                   how='left')
test[:3]

In [ ]:
test.columns

In [ ]:
train.columns

In [ ]:
test = test.rename({'index':'item_id'})
test = test[["item_id", "reference",'n_clicks','views','clicks_views_ratio','position', 'mean_price_difference',
               'Satisfactory Rating','Good Rating','Very Good Rating', 'Excellent Rating',
                '1 Star','2 Star','3 Star','4 Star','5 Star',
              ]]
test = test.drop(columns=['reference'], axis=1)
test = test.dropna()
test[:3]

In [ ]:
test['Stars'] = test['1 Star']*1 + test['2 Star']*2 + test['3 Star']*3 + test['4 Star']*4 + test['5 Star']*5

test['Rating'] = 0
test.loc[test['Satisfactory Rating']==1, 'Rating'] = 1
test.loc[test['Good Rating']==1, 'Rating'] = 2
test.loc[test['Very Good Rating']==1, 'Rating'] = 3
test.loc[test['Excellent Rating']==1, 'Rating'] = 4


test[:3]

In [ ]:
X_test = test[['mean_price_difference','n_clicks','clicks_views_ratio','position','Rating','Stars']]
X_test = sc.transform(X_test)

In [ ]:
y_pred = logreg.predict_proba(X_test)

In [ ]:
y_pred

In [ ]:
result = pd.concat([index_df, pd.DataFrame(y_pred)], axis=1, sort=False)
# result = result.drop([0], axis=1)
result[:10]

In [ ]:
result = result.assign(impressions=lambda x: x["impressions"].apply(str))
result = result.sort_values(["user_id", "session_id", "timestamp", "step", 1],
                     ascending=[True, True, True, True, False])
result[:10]

In [ ]:
result.dtypes

In [ ]:
def group_concat(df, gr_cols, col_concat):
    """Concatenate multiple rows into one."""

    df_out = (
        df
        .groupby(gr_cols)[col_concat]
        .apply(lambda x: ' '.join(x))
        .to_frame()
        .reset_index()
    )
    return df_out

In [ ]:
result = group_concat(result, ["user_id","session_id","timestamp","step"], "impressions")
result = result.rename(columns={"impressions":"item_recommendations"})
result[:10]

In [ ]:
result.to_csv("D:\\Dokumenty\\Systemy_rekomendacyjne\\additional_resources\\2019-master\\data\\submission_ML.csv", index=False,)